# Music Generation using an LSTM

#### Final Project for Deep Learning (CS 7643)

By Daeil Cha, Daniel Dias, Chitwan Kaudan

### Global Variables

In [10]:
# data_path = "../../lmd_matched"
data_path = "../clean-data"
saved_models_path = "../saved-models"

num_epochs = 10 # 1000
batch_size = 10
num_time_steps = 256
num_total_songs = 500

### Environment

In [11]:
import os

import torch

import numpy as np
import matplotlib.pyplot as plt

from IPython.core.debugger import set_trace
from getdata import getBatch

%matplotlib inline

%load_ext autoreload
%autoreload 2

learning_rate = 1e-6

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##### Pytorch GPU/CPU

In [12]:
dtype = torch.FloatTensor
device = torch.device("cpu")

# dtype = torch.cuda.DoubleTensor
# device = torch.device("cuda:0")

print(device)

cpu


### Data

#### Load In Data

In [13]:
start = 0
all_data = []

while start < num_total_songs:
    batch = np.array(getBatch(start, batch_size, num_time_steps, data_path), dtype='double')
    all_data.append(batch)
    # Shape should be (batch_size x num_time_steps x note_range x pitch/articulation)
    start += batch_size

all_data = np.concatenate(all_data, axis=0)

In [14]:
all_data.shape

(500, 256, 78, 2)

#### Apply Input Kernel

In [15]:
from model.input_function import InputKernel
inputkernel = InputKernel.apply

note_state_batch = torch.from_numpy(np.swapaxes(all_data,1,2)).float() 
#input kernel expects input shape = batch_size x num_notes x num_timesteps x 2
midi_high = 101
midi_low = 24
time_init=0
note_state_batch = inputkernel(note_state_batch,midi_low,midi_high,time_init)

note_state_batch.shape
#input kernel's output shape = batch_size x num_notes x num_timesteps x 80

torch.Size([500, 78, 256, 80])

#### Partition Data

In [5]:
x_train = None
y_train = None

x_val = None
y_val = None

x_test = None
y_test = None

all_expected = np.empty(all_data.shape)
all_expected[:, 0:all_expected.shape[1]-1] = all_data[:, 1:all_data.shape[1]]
all_expected[:, all_expected.shape[1]-1] = 0

all_data = np.reshape(all_data, (num_total_songs, num_time_steps, -1))
all_expected = np.reshape(all_expected, (num_total_songs, num_time_steps, -1))

print("all data:", all_data.shape)
print("all expected:", all_expected.shape)

orig_dataset = torch.utils.data.TensorDataset(torch.from_numpy(all_data).type(dtype), torch.from_numpy(all_expected).type(dtype))
x_train, x_test = torch.utils.data.random_split(orig_dataset, [450, 50])

x_train_loader = torch.utils.data.DataLoader(x_train, batch_size=batch_size, shuffle=True)
x_test_loader = torch.utils.data.DataLoader(x_test)

all data: (500, 256, 156)
all expected: (500, 256, 156)


## Functions

#### Train Steps

In [6]:
def train_step(x, y, model, loss_criterion, optimizer):
    y_pred = model(x)

    # Compute and print loss
    # loss = loss_criterion(torch.max(y_pred, dim=1).indices, y)
    loss = loss_criterion(y_pred, y)
    ret_val = loss.item()

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return ret_val

def test_step(x, y, model, loss_criterion):
    predictions = model(x)
    loss = loss_criterion(y_pred, y)
    
    return loss.item()

#### Save/Load Model

In [8]:
def save_model(model, model_name):
    torch.save(model.state_dict(), os.path.join(saved_models_path, "/", model_name))

def load_model_parameters(model, model_name):
    model.load_state_dict(torch.load(os.path.join(saved_models_path, "/", model_name)))

def load_new_model(model_name, model_constructor, *args):
    model = model_constructor(args)
    load_model_parameters(model, model_name)
    return model

### Model

In [30]:
from model.main_model import MusicGeneration

model = MusicGeneration(time_sequence_len=num_time_steps, batch_size=batch_size, time_hidden_size=36, data_type=dtype)

loss_criterion = torch.nn.MSELoss(reduction='sum') # = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#### Move To Correct Device

In [26]:
model.to(device)

MusicGeneration(
  (lstm_time0): LSTM(156, 36, batch_first=True)
  (lstm_note0): LSTM(256, 156, batch_first=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

## Training

In [ ]:
for epoch in range(num_epochs):
    print("Epoch:", epoch)
    for i, data in enumerate(x_train_loader, 0):
        x, y = data[0].to(device), data[1].to(device)

        loss = train_step(x, y, model, loss_criterion, optimizer)

        if i % 100 == 0:
            print(" -", loss)

## Test

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        x, y = data[0].to(device), data[1].to(device)

        loss = test_step(x, y, model, loss_criterion)

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

##### IPDB Test Code

In [31]:
set_trace()

def test_code():
    set_trace()

    train_iter = iter(x_train_loader)
    data = next(train_iter)
    x, y = data[0].to(device), data[1].to(device)

    loss = train_step(x, y, model, loss_criterion, optimizer)

test_code()

--Return--
None
> <ipython-input-31-1d033bb601ef>(1)<module>()
----> 1 set_trace()
      2 
      3 def test_code():
      4     set_trace()
      5 

ipdb> c
> <ipython-input-31-1d033bb601ef>(6)test_code()
      4     set_trace()
      5 
----> 6     train_iter = iter(x_train_loader)
      7     data = next(train_iter)
      8     x, y = data[0].to(device), data[1].to(device)

ipdb> n
> <ipython-input-31-1d033bb601ef>(7)test_code()
      5 
      6     train_iter = iter(x_train_loader)
----> 7     data = next(train_iter)
      8     x, y = data[0].to(device), data[1].to(device)
      9 

ipdb> n
> <ipython-input-31-1d033bb601ef>(8)test_code()
      6     train_iter = iter(x_train_loader)
      7     data = next(train_iter)
----> 8     x, y = data[0].to(device), data[1].to(device)
      9 
     10     loss = train_step(x, y, model, loss_criterion, optimizer)

ipdb> n
> <ipython-input-31-1d033bb601ef>(10)test_code()
      8     x, y = data[0].to(device), data[1].to(device)
      9 
--->

C:\Users\Daniel\Anaconda3\envs\cs7643music\lib\site-packages\torch\nn\modules\loss.py:431: UserWarning: Using a target size (torch.Size([10, 256, 156])) that is different to the input size (torch.Size([10, 36, 156])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (36) must match the size of tensor b (256) at non-singleton dimension 1
> <ipython-input-31-1d033bb601ef>(10)test_code()
      8     x, y = data[0].to(device), data[1].to(device)
      9 
---> 10     loss = train_step(x, y, model, loss_criterion, optimizer)
     11 
     12 test_code()

ipdb> print(loss)
*** NameError: name 'loss' is not defined
ipdb> continue


RuntimeError: The size of tensor a (36) must match the size of tensor b (256) at non-singleton dimension 1